# In this Notebook we will learn how we can use different documents and get them analysed by GPT

## First of all we will install the required packages that will help us along the way
1. openai: This package will help us to call the chat completion method of openai to generate results using GPT. Here we have used GPT-3.5 deployed in Azure portal
2. python-dotenv: This is used to load all the keys in the environment from a .env file
3. PyMuPDF: This package is used for easy PDF manipulation.
4. tiktoken: This package is used to calculate the tokens in a text

In [1]:
%%capture
!pip install openai==1.3.9 python-dotenv==1.0.0 PyMuPDF==1.24.2 PyMuPDFb==1.24.1 tqdm tiktoken

### Create a .env file in this folder with the following keys in it.

* AZURE_OPENAI_ENDPOINT
* AZURE_OPENAI_API_KEY
* AZURE_OPENAI_ENDPOINT
* AZURE_OPEN_AI_MODEL
* OPENAI_API_TYPE

In [2]:
import openai
from dotenv import load_dotenv
import fitz
from tqdm import tqdm
import os
import tiktoken

load_dotenv()

# os.environ["OPENAI_API_KEY"] = "<Place you key here>"
# model_name = "gpt-3.5-turbo"

os.environ["OPENAI_API_TYPE"] = "<Place you value here>"
os.environ["AZURE_OPENAI_ENDPOINT"] = "<Place you value here>"
os.environ["AZURE_OPENAI_API_KEY"] = "<Place you key here>"
os.environ["OPENAI_API_VERSION"] = "<Place you value here>"

model_name = "<Place you value here>"

token_encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

Here we have created a function that is called for generating response from OpenAI

This loads the keys from the environment and uses it to call the `openai.chat.completions.create` method

This method takes the system message and the user message as input for generating response.

### The temperature parameter defines the randomness of the output,

Higher the temperature the more creative the LLM will become with its answers, thus higher temperatures are used for poem generation, jokes etc.
Lower temperature gives deterministic results and return the most probable next token.

### Top P 

A sampling technique with temperature, called nucleus sampling, where you can control how deterministic the model is. If you are looking for exact and factual answers keep this low. If you are looking for more diverse responses, increase to a higher value. If you use Top P it means that only the tokens comprising the top_p probability mass are considered for responses, so a low top_p value selects the most confident responses. This means that a high top_p value will enable the model to look at more possible words, including less likely ones, leading to more diverse outputs.

In [3]:
def CallOpenAI(user,system):
  response = openai.chat.completions.create(
              model= model_name, # model = "deployment_name".
              temperature= 0,
              top_p= 0,
              messages=[
                  {"role": "system", "content": system},
                  {"role": "user", "content": user}
              ]
          )
  return response

## Lets take a contract and try to analyse it without much instruction

First we load the PDF and extract the texts from it and generate the token count of the text

In [21]:
def extract_text(pdf_path):
  pdf = fitz.open(pdf_path)
  text = ''

  for page in pdf:
    text += page.get_text()

  num_tokens = len(token_encoding.encode(text))
  print("Number of tokens in the entire Document: ", num_tokens)
  return text

Out here we can see the token count of the document is 11590 which is well withing the 16000 context limit of the GPT-3.5 model

In [22]:
short_document = extract_text("/content/AWS1.pdf")

Number of tokens in the entire Document:  11590


## We concatenate the text from the PDF and the question that the user wants to ask to the GPT about the PDF and form a prompt that we will use to generate the response using `openai.chat.completion.create` method

In [23]:
Question = "What is the governing courts for Amazon Web Services South Africa ProprietaryLimited"

full_prompt_SD = "<Context>"+short_document+"</Context>" +"\n\n" +"<Question>"+Question+"</Question>"

In [24]:
response = CallOpenAI(full_prompt_SD,"You are a Professional lawyer who can analyse documents thorougly")

### We can see that the GPT was able to generate the answer by refering to the prompt and give the correct result

In [25]:
print(response.choices[0].message.content)

The governing courts for Amazon Web Services South Africa Proprietary Limited would be the South Gauteng High Court in Johannesburg, South Africa.


## Now lets load up a document that has more than 16000 tokens, which is the limit of GPT-3.5-Turbo

In [26]:
long_document = extract_text("/content/PROFRAC HOLDINGS, LLC credit agreement.pdf")

Number of tokens in the entire Document:  163227


In [27]:
Question = "What is the Acknowledgement Regarding Any Supported QFCs?"

full_prompt_LD = "<Context>"+long_document+"</Context>" +"\n\n" +"<Question>"+Question+"</Question>"

## Here what you see is, when the message length exceeded the limit of GPT, it throws an error.
### This problem will be fixed in the next lab where you see how Retrieval Augmented Generation(RAG) will fix this problem and enable us to analyse documents of any length.

In [28]:
try:
  response = CallOpenAI(full_prompt_LD,"You are a Professional lawyer who can analyse documents thorougly")
except Exception as e:
  print(str(e))

Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, your messages resulted in 163263 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
